<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/Relation%20Extraction/5.AutomatedRelationExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebook'
os.symlink('/content/drive/Shared drives/BigDATA TEAM 10', nb_path)
sys.path.insert(0,nb_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/mnt


In [ ]:
from konlpy.tag import Hannanum, Okt
import re
import time
import numpy as np
import heapq
import pandas as pd
from operator import itemgetter
from collections import deque
from ast import literal_eval
from collections import defaultdict
from soykeyword.lasso import LassoKeywordExtractor
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
import kss
%cd /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction
from frameBERT import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
h = Hannanum()
okt = Okt()

In [ ]:
text = '''
평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. 인간은 가령 인종이나 성별과 상관없이 누구나 평등하다고 생각한다. 모든 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까? 그리고 그 근거는 무엇인가? 
일단 이 말을 모든 인간을 모든 측면에서 똑같이 대우하는 절대칙 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 대어난 능력과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. 평등에 대한 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한 이유가 제시되지 않은 불평등을 제거하는 데 목표를 두고 있다. ‘이유 없는 차별 금지’라는 조건적 평등 원칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하는 사람들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별저 대우를 하는 것을 허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근거에 대한 물음이다.
근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난 자연적 권리로 간주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다. 그래서 롤스는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런하려고 한다. 그는 어떤 규칙이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적 정의는 실현된다고 본다. 하지만 롤스는 형식저 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이 더 높은 도덕적 권위를 지닌 다른 이넘과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다.
롤스는 인간 평등의 근거를 설명하면서 영역 성질 (range property) 개념을 도입한다. 예를 들어 어떤 원의 내부에 있는 점들은 그 위치가 서로 다르지만 원의 내부에 있다는 점에서 동일한 영역 성질을 갖는다. 반면에 원의 내부에 있는 집과 원의 외부에 였는 점은원의 경계선을 기준으로 서로 다른 영역 성질을 갖는다. 그는 평등한 대우를 받기 위한 영역 성질로서 ‘도덕적 인격'을 제시한다. 도덕적 인격이란 도덕적 호소가 가능하고 그런 호소에 관심을 기이는 능력이 있다는 것인데, 이 능력을 최소치만 갖고 있다면 평등한 대우에 대한 권한을 갖게 된다. 도덕적 인격이라고 해서 도덕적으로 훌륭하다는 뜻이 아니라 도덕과 무관하다는 말과 대비되는 뜻으로 쓰고 있다. 그런데 어린 아이는 인격체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다. 이에 대해 롤스는 도덕적 인격을 규정하는 최소한의 요구 조건은 잠재적 능력이지 그것의 실현 여부가 아니기에 어린 아이도 평등한 존재라고 말한다. 싱어는 위와 같은 롤스의 시도를 비판한다. 도덕에 대한 민감성의 수준은 사람에 따라 다르다. 그래서 도덕적 인격의 능력이 그렇게 중요하다면 그것을 갖춘 정도에 따라 도덕적 위계를 다르게 하지 말아야 할 이유가 분명하지 않다고 말한다. 
'''

In [ ]:
paragraphs = text.split('\n')
paragraphs = [i for i in paragraphs if i]
wordrank_extractor = KRWordRank(min_count=4, max_length=10)
keywords, rank, graph = wordrank_extractor.extract(paragraphs, num_keywords=30)
print(keywords)

In [ ]:
class Text:
  def __init__(self, text):
    self.text = text
    text = re.sub('\n',' ',text)
    text = re.sub("'",' ',text)
    paragraphs = text.split('\n')
    self.paragraphs = [i for i in paragraphs if i]
    self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]

  def keywords(self, num = 40):
    wordrank_extractor = KRWordRank(min_count=4, max_length=10)
    keywords, rank, graph = wordrank_extractor.extract(self.paragraphs, num_keywords=num)
    p=[]
    kw=[]
    for k, v in keywords.items():
      p.append(okt.pos(k))
    for ls in p:
      for tup in ls:
          if tup[1].startswith("N"):
            kw.append(tup[0])
    return kw
    
class Highlight(Text):
  def __init__(self, text, candidates=None):
    self.text = text
    self.newtext = deepcopy(self.text)
    if candidates==None:
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록'
        conj = conj.replace("'","")
        conj = conj.replace(", "," ")
        self.candidates = conj.split(" ")
        self.idx = [(self.text.find(" "+cand), len(cand)) for cand in self.candidates if self.text.find(cand)!=-1]
    else:
        self.candidates = candidates
        self.idx = [(self.text.find(" "+cand), len(cand)) for cand in self.candidates if self.text.find(cand)!=-1]
  def add_tags(self, tag):
    for i in range(len(self.idx)):
      idx = [(self.newtext.find(" "+cand), len(cand)) for cand in self.candidates if self.newtext.find(cand)!=-1]
      indices = (idx[i][0],idx[i][0]+idx[i][1]+1)
      word = self.newtext[indices[0]+1:indices[1]]
      if candidates = None:
        tagged = "<mark style='background-color: #0F4C81'>%s</mark>" % (tag, word, tag)
      else:
        tagged = "<mark style='background-color: #FF6F61'>%s</mark>" % (tag, word, tag)
      self.newtext = tagged.join([self.newtext[:indices[0]], self.newtext[indices[1]:]])
    return self.newtext


In [ ]:
def findall(p, s):
    i = s.find(p)
    while i != -1:
        yield i
        i = s.find(p, i+1)

[(i, i+len("하지만")) for i in findall("하지만", text)]

[(590, 593), (782, 785)]

In [ ]:
def main():
  t = Text(text)
  
  kw=t.keywords()
  print(kw)
  h = Highlight(text, kw)
  tag = "keyword"
  css_format = h.add_tags(tag)
  print(css_format)
  h = Highlight(text)
  tag = "conj"  # html 만들 떄 mark 부분을 conj로 변경하기
  css_format=h.add_tags(tag)
  print(css_format)

In [ ]:
main()

In [ ]:
pprint(okt.tagset)
print()
pprint(h.tagset)

#### 하지만, 그런데 등의 연결사에 해당하는 부분 추출

css 문법
<html>
<head>
<style>
mark { 
  background-color: blue;
  color: black;
}
</style>
</head>
<body>

<p>A mark element is displayed like this:</p>

평등은 자유와 더불어 근대 사회의 핵심 이념으로 자리 잡고있다. <mark>인간</mark>은 가령 인종이나 성별과 상관없이 누구나 <mark>평등</mark>하다고 생각한다.<keyword>모든</keyword> 인간은 평등하다고 말하는데, 이 말은 무슨 뜻일까? 그리고 그<keyword>근거</keyword>는<keyword>무엇</keyword>인가? 
일단 이 말을 모든 인간을 모든 측면에서 똑같이<keyword>대우</keyword>하는 절대칙 평등으로 생각하는 이는 없다. 인간은 저마다 다르게 가지고 대어난<keyword>능력</keyword>과 소질을 똑같게 만들 수 없기 때문이다. 절대적 평등은 개인의 개성이냐 자율성 등의 가치와 충돌하기도 한다. 평등에<keyword>대한</keyword> 요구는 모든 불평등을 악으로 보는 것이 아니라 충분한<keyword>이유</keyword>가<keyword>제시</keyword>되지 않은 불평등을 제거하는 데 목표를 두고 있다. ‘이유 없는 차별 금지’라는 조건적 평등<keyword>원</keyword>칙은 차별 대우를 할 때는 이유를 제시할 것을 요구하고 있다. 이것은 어떤 이유가 제시된다면 특정한 부류에 속하는<keyword>사람</keyword>들에게는 평등한 대우를, 그 부류에 속하지 않는 사람들에게는 차별저 대우를 하는 것을 허용한다. 그렇다면 사람들을 특정한 부류로 구분하는 기준은 무엇인가? 이것은 바로 평등의 근거에 대한 물음이다.
근대의 여러 인권 선언에 나타난 평등 개념은 개인들 사이의 평등성을 타고난<keyword>자연</keyword>적 권리로 간주하였다. 하지만 이러한 자연권 이론은 무엇이 자연적 권리이고 권리의 존재가 자명한 이유가 무엇인지 등의 문제에 부딪히게 된다. 그래서<keyword>롤스</keyword>는 기존의 자연권 사상에 의존하지 않는 방시으로 인간 평등의 근거를 마런하려고 한다. 그는 어떤<keyword>규칙</keyword>이 공평하고 일관되게 운영되며, 그 규칙에 따라 유사한 경우는 유사하게 취급된다면 형식적<keyword>정의</keyword>는 실현된다고 본다. 하지만 롤스는 형식저 정의에 따라 규칙을 준수하는 것만으로는 정의를 담보할 수 없다고 생각한다. 그 규칙이 더 높은<keyword>도덕</keyword>적 권위를 지닌 다른 이넘과 충돌할 수 었기에, 실질적 정의가 보장되기 위해서는 규칙의 내용이 중요한 것이다.
롤스는 인간 평등의 근거를 설명하면서<keyword>영역</keyword><keyword>성질</keyword> (range property) 개념을 도입한다. 예를 들어 어떤 원의 내부에 있는 점들은 그 위치가 서로 다르지만 원의 내부에 있다는 점에서 동일한 영역 성질을 갖는다. 반면에 원의 내부에 있는 집과 원의 외부에 였는 점은원의 경계선을 기준으로 서로 다른 영역 성질을 갖는다. 그는 평등한 대우를 받기 위한 영역 성질로서 ‘도덕적<keyword>인격</keyword>'을 제시한다. 도덕적 인격이란 도덕적 호소가 가능하고 그런 호소에 관심을 기이는 능력이 있다는 것인데, 이 능력을 최소치만 갖고 있다면 평등한 대우에 대한 권한을 갖게 된다. 도덕적 인격이라고 해서 도덕적으로 훌륭하다는 뜻이 아니라 도덕과 무관하다는 말과 대비되는 뜻으로 쓰고 있다. 그런데 어린 아이는 인격체로서의 최소한의 기준을 충족하고 있는지가 논란이 될 수 있다. 이에 대해 롤스는 도덕적 인격을 규정하는 최소한의 요구 조건은 잠재적 능력이지 그것의 실현 여부가 아니기에 어린 아이도 평등한 존재라고 말한다. 싱어는 위와 같은 롤스의 시도를 비판한다. 도덕에 대한 민감성의 수준은 사람에 따라 다르다. 그래서 도덕적 인격의 능력이 그렇게 중요하다면 그것을 갖춘 정도에 따라 도덕적 위계를 다르게 하지 말아야 할 이유가 분명하지 않다고 말한다. 


<p>Change the default CSS settings to see the effect.</p>

</body>
</html>


## Relation Extraction

In [ ]:
parser = frame_parser.FrameParser(model_path=path, language='ko')

### Linked List Implemented to Show the whole list of sentences and their parsed tags in a consecutive manner

In [ ]:
def split_into_sentences(object):
  ls = kss.split_sentences(object)
  return ls

def frameParse(text):
  parser = frame_parser.FrameParser(model_path=path, language='ko')
  parsed = parser.parser(text, sent_id='1', result_format='conll')
  return parsed
  

def sortCandidate(parsed, num_candidates):
  for i in range(len(parsed)):
    count=0
    for element in parsed[i][3]:
      if element.startswith("O"):
        count+=1
  q[i] = len(words) - count
  hq = heapq.nlargest(num_candidates, q)
  indices.append(q.index(hq[idx]))
  # for idx in range(num_candidates):
  #   temp = parsed[q.index(hq[idx])]
  #   words = temp[0]
  #   [[row[i] for row in temp] for i in range(len(words))]
  return indices


def findConsecutiveBIO(words, tag):
  began = False
  count = 0
  q = deque(words)
  for i in range(len(words)):
    if tag[i]=='O':
      q.popleft()
    if tag[i].startswith('B'):
      began=True
    if began & tag[i].startswith('I'):
      count+=1
  return q

# Linked List 구현해서 parsed 후보군에 사용
class Node:
    def __init__(self, data):
        self.words = data[0]
        self.tags = data[3]
        self.next = None

    def __repr__(self):
        return str((self.words, self.tags))

class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)
  
class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)

def extractFrame(text):
  ls = split_into_sentences(text)
  final = {}
  
  for idx in range(len(ls)):
    parsed = frameParse(ls[idx]) # candidates 생성
    final.setdefault(idx,str)
    parsedList =  LinkedList()
    for j in range(len(parsed)):
      parsed_candidate = parsed[j]
      new_node = Node(parsed_candidate)
      if j == 0:
        old_node = new_node
        parsedList.head = old_node
      elif j==len(parsed)-1:
        old_node.next = new_node
        new_node.next = None
        print(idx,'  ',parsedList)
        final[idx] = parsedList
      else:
        old_node.next = new_node
        old_node = new_node
  return final



In [ ]:
def main(text):
  all = extractFrame(text)
  s = ""
  res = {}

  i = 1
  res.setdefault(i,tuple)
  for k, v in all.items():
    a = v.head
    while a is not None:
      temp = " "
      if len(a.words) == len(a.tags):
        q = findConsecutiveBIO(a.words, a.tags)
      else:
        print("ERROR OCCURED")
        
      count = len(a.words)
      for tag in a.tags:
       if tag.startswith("O"):
          count -= 1
      if count > len(a.words)*0.5:
        print("The threshold is reached !! 😛 😝 😜 🤪 🤨 🧐 🤓 😎 🤩 🥳 😏")
        temp = " ".join(q) 
        print(temp)
        words = a.words
        tags = a.tags
      res[i] = (a.words, a.tags , " ".join(q))
      a = a.next
    i+=1
  return res


In [ ]:
extracted = main(text)
pprint(extracted)

In [ ]:
pprint(extracted)

In [ ]:
for k, tup in extracted.items():
  a, b, c = tup
  roles = [0]*len(b)
  words = [0]*len(b)
  for i in range(len(b)):
    try: 
      roles[i] = b[i].split("-")[1] 
      
    except: 
      roles[i] = b[i].split("-")[0]
  print(roles)
  for _ in roles:
    pass

In [ ]:
# 1st sentence : GOALS & MEANS 

parsed = parser.parser(ls[0], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count
hq = heapq.nlargest(2, q)


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(2, q)) # 첫번째 경우 사용


words = parsed[0][0]
roles = parsed[0][2]
tagged = parsed[0][3]
for idx in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]

print("roles ",roles)

goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)

In [ ]:
# 2nd sentence : PURPOSE & INSTRUMENT
parsed = parser.parser(ls[1], sent_id='1', result_format='conll')
words = parsed[0][0]
print("😀 Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count


print("Number of parsed candidates ",len(parsed))
print(q, heapq.nlargest(3, q))
print(itemgetter(*[1,5])(parsed))
candidates = itemgetter(*[1,8])(parsed)
for _ in candidates:
  words = _[0]
  tagged = _[3]
  print("tagged vector of candidates: ",tagged)
  roles = _[2]
  for i in range(len(parsed)):
    try:
      roles[i] = tagged[i].split("-")[1]
    except:
      roles[i] = tagged[i].split("-")[0]
  print(roles,f'Tagged words for the roles : {tagged}')
# roles are selected as the one with purpose and instrument

purpose = []
instrument = []
for _ in range(len(words)):
  if roles[_]=="Purpose":
    purpose.append(words[_])
    
  elif roles[_]=="Instrument":
    instrument.append(words[_])
    
  elif roles[_]!="_": # using 포함
    instrument.append(words[_])
print("Instrument: ",instrument)
print("Purpose: ",purpose)





# PURPOSE
PURPOSE = ' '.join(purpose)
pos = h.pos(PURPOSE)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
PURPOSE_TRUNCATED = ' '.join(imp)
print(PURPOSE_TRUNCATED)

# INSTRUMENT
INSTRUMENT = ' '.join(instrument)
pos = h.pos(INSTRUMENT)
imp = []
for tup in pos:
  if tup[1]=="N":
    imp.append(tup[0])
INST_TRUNCATED = ' '.join(imp)
print(INST_TRUNCATED)

final_string = INST_TRUNCATED+ ' ➡️' + PURPOSE_TRUNCATED 
print("😀 final string to be inserted ", final_string)

    
    

In [ ]:
# 3rd sentence : Goal - Means
parsed = parser.parser(ls[2], sent_id='1', result_format='conll')
words = parsed[0][0]
print("Words vector from sentence ",words)
q = [-1]*len(parsed)
for i in range(len(parsed)):
  count=0
  for element in parsed[i][3]:
    if element.startswith("O"):
      count+=1
  q[i] = len(words) - count

print(q, heapq.nlargest(3, q))

a,b,c=parsed
tagged = b[3]
for i in range(len(tagged)):
  try:
    roles[i] = tagged[i].split("-")[1]
  except:
    roles[i] = tagged[i].split("-")[0]



goals = []
means = []
for _ in range(len(words)):
  if roles[_]=="Goal":
    goals.append(words[_])
  if roles[_]=='Means':
    means.append(words[_])

# MEANS
MEANS = ' '.join(means)
pos = h.pos(MEANS)
imp = []
for tup in h.pos(MEANS):
  if tup[1]=="N":
    imp.append(tup[0])
MEANS_TRUNCATED = ' '.join(imp)
print(MEANS_TRUNCATED)

# GOALS
GOAL = ' '.join(goals)
pos = h.pos(GOAL)
imp = []
for tup in h.pos(GOAL):
  if tup[1]=="N":
    imp.append(tup[0])
GOAL_TRUNCATED = ' '.join(imp)
print(GOAL_TRUNCATED)

final_string = MEANS_TRUNCATED + ' -- >' + GOAL_TRUNCATED
print("😀 final string to be inserted",final_string)
